In [1]:
import torch
import net

In [2]:
device = torch.device('cuda:0')
batch_size = 2
img_size = 256
path = ('YSGR/*.jpg', 'dataB/*.jpg')
lr = 0.0001
betas = (0, 0.999)
light = False
weight = [1,10,10,1500,0,20]
weight_decay = 0.0001
ID = 1
total_iteration = 150000

In [3]:
Net = net.UGATIT(device = device, batch_size = batch_size, img_size = img_size, path = path,
           lr = lr, betas = betas, light = light, weight = weight, weight_decay = weight_decay,
           ID = ID, total_iteration = total_iteration
          )

In [4]:
Net.load('lastest')

--------------------------
      125K iter Load 
--------------------------


In [ ]:
Net.train()

[125/150] [ModelA G : 4.486 | D : 0.723] [ModelB G : 4.476 | D : 0.784]: 100%|█████| 1000/1000 [19:59<00:00,  1.20s/it]
[126/150] [ModelA G : 4.208 | D : 0.986] [ModelB G : 4.571 | D : 0.686]: 100%|█████| 1000/1000 [20:16<00:00,  1.22s/it]
[127/150] [ModelA G : 4.216 | D : 0.724] [ModelB G : 4.256 | D : 1.295]: 100%|█████| 1000/1000 [20:30<00:00,  1.23s/it]
[128/150] [ModelA G : 4.080 | D : 1.095] [ModelB G : 3.562 | D : 1.056]: 100%|█████| 1000/1000 [19:37<00:00,  1.18s/it]
[129/150] [ModelA G : 3.711 | D : 0.988] [ModelB G : 5.153 | D : 1.232]: 100%|█████| 1000/1000 [19:33<00:00,  1.17s/it]
[130/150] [ModelA G : 4.593 | D : 1.157] [ModelB G : 3.962 | D : 1.394]: 100%|█████| 1000/1000 [19:45<00:00,  1.19s/it]
[131/150] [ModelA G : 3.898 | D : 1.084] [ModelB G : 3.692 | D : 1.462]: 100%|█████| 1000/1000 [19:34<00:00,  1.17s/it]
[132/150] [ModelA G : 3.703 | D : 1.342] [ModelB G : 3.714 | D : 1.194]: 100%|█████| 1000/1000 [19:33<00:00,  1.17s/it]
[133/150] [ModelA G : 3.990 | D : 1.417]

In [9]:
import os
import glob

In [12]:
os.rename

<function nt.rename(src, dst, *, src_dir_fd=None, dst_dir_fd=None)>

In [38]:
A = glob.glob('1/model/*.pt')
for i in range(2,120):
    name = '1/model/{:03}_k_model.pt'.format(i)
    if not name in A:
        continue
    if i % 10 == 1:
        os.rename(name, '1/model/{:03}_k_model.pt'.format(i-1))
    else:
        os.remove(name)

In [42]:
A = glob.glob('1/model/*.pt')



In [30]:
os.rename('1/1234.txt', '1/12345.txt')

In [28]:
os.rename(name, 'D:\\Jupyter\\AICHALLENGE/1/model/102_k_model_.pt')

FileNotFoundError: [WinError 2] 지정된 파일을 찾을 수 없습니다: 'D:\\Jupyter\\AICHALLENGE/1/model/102_k_model.pt' -> 'D:\\Jupyter\\AICHALLENGE/1/model/102_k_model_.pt'

In [43]:
A

['1/model\\010_k_model.pt',
 '1/model\\020_k_model.pt',
 '1/model\\030_k_model.pt',
 '1/model\\040_k_model.pt',
 '1/model\\050_k_model.pt',
 '1/model\\060_k_model.pt',
 '1/model\\070_k_model.pt',
 '1/model\\080_k_model.pt',
 '1/model\\090_k_model.pt',
 '1/model\\110_k_model.pt',
 '1/model\\111_k_model.pt',
 '1/model\\112_k_model.pt',
 '1/model\\113_k_model.pt',
 '1/model\\114_k_model.pt',
 '1/model\\115_k_model.pt',
 '1/model\\116_k_model.pt',
 '1/model\\117_k_model.pt',
 '1/model\\118_k_model.pt',
 '1/model\\119_k_model.pt',
 '1/model\\120_k_model.pt',
 '1/model\\121_k_model.pt',
 '1/model\\122_k_model.pt',
 '1/model\\123_k_model.pt',
 '1/model\\124_k_model.pt',
 '1/model\\125_k_model.pt',
 '1/model\\126_k_model.pt',
 '1/model\\lastest_model.pt']

In [41]:
name

'1/model/119_k_model.pt'